In [5]:
import numpy as np
import os
import matplotlib.pyplot as plt
import scipy as spy
import librosa
import random
# import tensorflow as tf

In [2]:
FOLDER_PATH = os.path.join("train", "audio")

using a sampling rate of 16000, we will first check the number of files corresponding to each level

In [3]:
NYQUIST_RATE = 8000
labels = ["yes", "no", "up", "down", "left", "right", "on", "off", "stop", "go", "silence", "unknown"]
unknown = []
len_unknown = 0

for i in os.listdir(FOLDER_PATH):
    if i == '_background_noise_':
        continue
    if i in labels :
        if i == "silence" :
            continue
        if i != "unknown" :
            print(i, len(os.listdir(os.path.join(FOLDER_PATH, i))))
        else : 
            continue
    else :
        unknown.append(i)
        len_unknown += len(os.listdir(os.path.join(FOLDER_PATH, i)))
print(len_unknown)
print(unknown)


down 2359
go 2372
left 2353
no 2375
off 2357
on 2367
right 2367
stop 2380
up 2375
yes 2377
41039
['bed', 'bird', 'cat', 'dog', 'eight', 'five', 'four', 'happy', 'house', 'marvin', 'nine', 'one', 'seven', 'sheila', 'six', 'three', 'tree', 'two', 'wow', 'zero']


Now we will sample all the training files to the Nyquist rate and then store the data in label.npy file

In [14]:
import random

for i in os.listdir(FOLDER_PATH) :
#     print(i)
    if i in ["silence", "unknown", "_background_noise_"]:
        continue
    elif i not in unknown :
#         print(i, " is in folder path")
        label_path = os.path.join(FOLDER_PATH , i)
        x_train = (os.listdir(label_path))
        samples = []
        for j in x_train:
            x, sampling_rate = librosa.load(os.path.join(label_path,j), 16000)
            x = librosa.util.fix_length(x, 16000)
            samples.append(x)
        print(len(samples))
        print(len(samples[0]))
        samples = np.array(samples)
        np.save(i,samples)
        mat = np.load(i + ".npy")
        print(mat.shape)

bed
bird
cat
dog
down
down  is in folder path
2359
16000
(2359, 16000)
eight
five
four
go
go  is in folder path
2372
16000
(2372, 16000)
happy
house
left
left  is in folder path
2353
16000
(2353, 16000)
marvin
nine
no
no  is in folder path
2375
16000
(2375, 16000)
off
off  is in folder path
2357
16000
(2357, 16000)
on
on  is in folder path
2367
16000
(2367, 16000)
one
right
right  is in folder path
2367
16000
(2367, 16000)
seven
sheila
six
stop
stop  is in folder path
2380
16000
(2380, 16000)
three
tree
two
up
up  is in folder path
2375
16000
(2375, 16000)
wow
yes
yes  is in folder path
2377
16000
(2377, 16000)
zero
_background_noise_
(4096, 16000)


for the unknown label, we will sample the rest of the labels and pick random files from them to make the unknown label more robust.

In [8]:
unknown_train = []
for x in unknown :
    label_path = os.path.join(FOLDER_PATH, x)
    x_train = random.sample(os.listdir(label_path), len(os.listdir(label_path))//10)
    for j in x_train :
        mat, sampling_rate = librosa.load(os.path.join(label_path, j), 16000)
#         print(x.shape)
        mat = librosa.util.fix_length(mat, 16000)
#         print(mat.shape)
        unknown_train.append(mat)
unknown_train = np.array(unknown_train)
np.save("unknown", unknown_train)
mat = np.load("unknown.npy")
print(mat.shape)

(4096, 16000)


Now we will focus on the silence label. For this label, we will use the background noise folder. We will first divide these files into 1 second clips and then use them to train the data.

In [7]:
from pydub import AudioSegment
from pydub import silence

c:\users\lalitesh\appdata\local\programs\python\python39\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [13]:
import IPython.display as ipd
from IPython.core.display import display
from pydub import AudioSegment
from pydub.playback import play

path_label = os.path.join(FOLDER_PATH , "_background_noise_")
silence_train = []
for i in os.listdir(path_label):
    if( i == "README.md") :
        continue
    x, sampling_rate = librosa.load(os.path.join(path_label, i), sr=16000)
    
    file = ipd.Audio(os.path.join(path_label, i))
    dur = (librosa.get_duration(y = x, sr = sampling_rate))
    print(dur)
    samples_num = int(dur) 
    print(samples_num)
    print(x.shape)
    for_this_sample = []
    val = 0
    for i in range(0, samples_num):
        
        k = [x[i*16000 : (i+1)*16000]]
#         display(ipd.Audio(k, rate=16000))
        val += 1
        print(k, " ", val)
        silence_train += k
#     for_this_sample = random.sample(for_this_sample, len(for_this_sample))
print(len(silence_train))
np.save("silence", silence_train)
k = np.load("silence.npy")
print(k.shape)

train\audio\_background_noise_\doing_the_dishes.wav
95.183125
95
(1522930,)
[array([ 0.        ,  0.        ,  0.        , ..., -0.00158691,
       -0.02734375, -0.04486084], dtype=float32)]   1
[array([-0.06219482, -0.05032349, -0.0894165 , ...,  0.02548218,
        0.01483154, -0.02365112], dtype=float32)]   2
[array([-0.04998779, -0.02090454,  0.03244019, ...,  0.02630615,
        0.03662109,  0.06005859], dtype=float32)]   3
[array([ 0.07693481,  0.04910278,  0.01699829, ...,  0.02050781,
        0.00466919, -0.02853394], dtype=float32)]   4
[array([-0.05096436, -0.07260132, -0.03811646, ..., -0.02389526,
       -0.0184021 ,  0.00497437], dtype=float32)]   5
[array([ 0.02420044,  0.05566406,  0.10592651, ...,  0.01950073,
        0.01171875, -0.02786255], dtype=float32)]   6
[array([-0.02957153, -0.04510498, -0.07540894, ...,  0.03604126,
       -0.02209473, -0.06100464], dtype=float32)]   7
[array([-0.0322876 , -0.00674438, -0.01916504, ..., -0.00405884,
       -0.00378418, -0.023